In [189]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

print("🔨 Tentando criar o banco de dados 'db_aviao'...")

# 1. Conecta no banco padrão 'postgres' (que sempre existe)
# Nota: Usamos as credenciais 'admin'/'admin' que você definiu no docker-compose
try:
    con = psycopg2.connect(
        user='admin', 
        password='admin', 
        host='localhost', 
        port=5432, 
        database='postgres' # Conecta no default para poder criar outros
    )
    
    # Necessário para criar banco de dados (não pode estar em transação)
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = con.cursor()

    # 2. Tenta criar o banco novo
    cur.execute("CREATE DATABASE db_aviao;")
    print("SUCESSO! Banco 'db_aviao' criado.")
    
except psycopg2.errors.DuplicateDatabase:
    print("O banco 'db_aviao' já existe (tudo certo).")
except Exception as e:
    print(f" Erro ao tentar criar banco: {e}")
finally:
    if 'con' in locals(): con.close()

🔨 Tentando criar o banco de dados 'db_aviao'...
O banco 'db_aviao' já existe (tudo certo).


In [190]:
%pip install pandas numpy sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


# ETL Bronze to Silver: Aviation Data

Este notebook realiza o processamento da camada **Bronze (Raw)** para a **Silver (Trusted)**.
O objetivo é padronizar tipos de dados, limpar inconsistências observadas na análise exploratória e enriquecer o dataset com novas features.

**Pipeline:**
1. Ingestão do CSV Bruto (`cp1252`).
2. Limpeza e Padronização de Colunas.
3. Tratamento de Tipos (Datas, Inteiros, Booleanos).
4. Engenharia de Atributos (Extração de Ano, Mês, Categorias).
5. Carga no Data Warehouse (PostgreSQL).

## 1. Configuração e Importações
Importação das bibliotecas essenciais para manipulação de dados e conexão com banco.

In [191]:
import pandas as pd
import numpy as np
import re
import os
import unicodedata
import psycopg2
from sqlalchemy import create_engine
from psycopg2.extras import execute_batch

# Configurações de exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)

In [192]:
import os

print(f"Diretório onde o script está rodando: {os.getcwd()}")

# O caminho exato baseado na estrutura de pastas
# ../ volta para 'Acidentes_aviao'
# Data_Layer entra na pasta (com underline)
# raw/dados_brutos.csv é o arquivo final
INPUT_FILE = '../Data_Layer/raw/dados_brutos.csv'

if os.path.exists(INPUT_FILE):
    print(f"SUCESSO! Arquivo encontrado em: {INPUT_FILE}")
else:
    print(f"AINDA NÃO ACHOU em: {INPUT_FILE}")
    # Última tentativa: verificar se por acaso não está na mesma pasta
    if os.path.exists('dados_brutos.csv'):
        INPUT_FILE = 'dados_brutos.csv'
        print(f"Achou na mesma pasta!")
    else:
        raise FileNotFoundError("Verifique se o arquivo 'dados_brutos.csv' está mesmo dentro de 'Data_Layer/raw'")

# Configuração do Banco de Dados
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'db_aviao',
    'user': 'admin',
    'password': 'admin'
}

DB_CONNECTION_STR = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

Diretório onde o script está rodando: c:\Users\ATA\OneDrive\Documentos\Acidentes_aviao\Transformer
SUCESSO! Arquivo encontrado em: ../Data_Layer/raw/dados_brutos.csv


## 2. Funções Auxiliares
Desenvolvemos funções modulares para tratar problemas específicos identificados na fase de Analytics, como caracteres especiais e normalização de texto.

### 2.1 Limpeza de Texto e Strings
Função para remover espaços extras e caracteres estranhos dos nomes de cidades e modelos.

In [193]:
def clean_text(text):
    """
    Remove espaços extras e normaliza caracteres.
    Ex: ' Cessna  ' -> 'Cessna'
    """
    if pd.isna(text) or text == '':
        return 'Unknown'
    
    # Normaliza unicode (remove acentos se necessário, aqui mantemos compatibilidade)
    text_norm = unicodedata.normalize('NFKC', str(text)).strip()
    return text_norm

### 2.2 Tratamento de Severidade (Regra de Negócio)
Padronização da coluna `Injury.Severity`. Muitas vezes ela vem como "Fatal(2)" e queremos apenas separar a categoria do número.

In [194]:
def parse_severity(text):
    """
    Ex: 'Fatal(2)' -> 'Fatal'
    Ex: 'Non-Fatal' -> 'Non-Fatal'
    """
    if pd.isna(text):
        return 'Unavailable'
    
    text = str(text).strip()
    if text.lower().startswith('fatal'):
        return 'Fatal'
    elif text.lower().startswith('non-fatal'):
        return 'Non-Fatal'
    elif text.lower().startswith('incident'):
        return 'Incident'
    
    return 'Unavailable'

## 3. Carregando os Dados Brutos
Iniciamos o ETL lendo o arquivo CSV original. Tratamos o encoding `cp1252` que é comum em arquivos antigos ou gerados por Excel/Windows.

In [195]:
print("="*80)
print("ETL BRONZE -> SILVER: AVIATION DATA")
print("="*80)

print("\nCarregando dados Bronze...")
# Tratamento de erro de encoding observado na analytics
try:
    df = pd.read_csv(INPUT_FILE, encoding='cp1252', low_memory=False)
except:
    df = pd.read_csv(INPUT_FILE, encoding='latin-1', low_memory=False)

print(f"   Carregado: {df.shape[0]:,} linhas x {df.shape[1]} colunas")

ETL BRONZE -> SILVER: AVIATION DATA

Carregando dados Bronze...
   Carregado: 88,889 linhas x 31 colunas


## 4. Renomeação e Seleção de Colunas
Padronizamos os nomes das colunas para `snake_case` (padrão de banco de dados) e selecionamos apenas as colunas definidas no Dicionário de Dados.

In [196]:
print("\nRenomeando colunas...")

# Mapa de De-Para conforme Dicionário de Dados
colunas_map = {
    'Event.Id': 'event_id',
    'Investigation.Type': 'investigation_type',
    'Accident.Number': 'accident_number',
    'Event.Date': 'event_date',
    'Location': 'location',
    'Country': 'country',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Airport.Code': 'airport_code',
    'Airport.Name': 'airport_name',
    'Injury.Severity': 'injury_severity_raw', # Mantemos a original temporariamente
    'Aircraft.damage': 'aircraft_damage',
    'Aircraft.Category': 'aircraft_category',
    'Registration.Number': 'registration_number',
    'Make': 'make',
    'Model': 'model',
    'Amateur.Built': 'amateur_built',
    'Number.of.Engines': 'number_of_engines',
    'Engine.Type': 'engine_type',
    'Total.Fatal.Injuries': 'total_fatal_injuries',
    'Total.Serious.Injuries': 'total_serious_injuries',
    'Total.Minor.Injuries': 'total_minor_injuries',
    'Total.Uninjured': 'total_uninjured',
    'Weather.Condition': 'weather_condition',
    'Broad.phase.of.flight': 'broad_phase_of_flight',
    'Report.Status': 'report_status',
    'Publication.Date': 'publication_date'
}

# Filtra colunas que realmente existem no CSV
cols_to_use = [c for c in colunas_map.keys() if c in df.columns]
df = df[cols_to_use].rename(columns=colunas_map)


Renomeando colunas...


## 5. Conversão e Limpeza de Tipos
Correção de tipos de dados:
1. **Datas:** Converter string para objeto datetime.
2. **Numéricos:** Preencher Nulos (NaN) com 0 para colunas de vítimas.
3. **Booleanos:** Converter 'Yes'/'No' para True/False.

In [197]:
print("\nConvertendo tipos de dados...")

# 1. Datas
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')

# 2. Numéricos (Vítimas) - Regra: Nulo = 0
cols_vitimas = ['total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured']
for col in cols_vitimas:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# 3. Numéricos (Motores) - Regra: Nulo = 1 (Assunção conservadora ou manter nulo)
df['number_of_engines'] = pd.to_numeric(df['number_of_engines'], errors='coerce').fillna(1).astype(int)

# 4. Lat/Long
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

print("   Conversão concluída.")


Convertendo tipos de dados...
   Conversão concluída.


C:\Users\ATA\AppData\Local\Temp\ipykernel_7460\737331554.py:5: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')


## 6. Engenharia de Atributos (Feature Engineering)
Criação de novas colunas derivadas para facilitar a análise no Power BI (Camada Gold).

In [198]:
print("\nCriando novas features...")

# 6.1 Tratamento de Severidade Limpa
df['injury_severity'] = df['injury_severity_raw'].apply(parse_severity)

# 6.2 Extração de Ano e Mês (Performance no Banco)
df['year'] = df['event_date'].dt.year
df['month'] = df['event_date'].dt.month

# 6.3 Flag de Acidente Fatal (Para KPIs rápidos)
df['is_fatal'] = df['total_fatal_injuries'] > 0

# 6.4 Tratamento de Amateur Built (Booleano)
# Remove espaços e converte para booleano real
df['amateur_built'] = df['amateur_built'].astype(str).str.lower().str.strip() == 'yes'

# 6.5 Limpeza de Texto Final
txt_cols = ['location', 'country', 'make', 'model', 'weather_condition']
for col in txt_cols:
    df[col] = df[col].apply(clean_text)

print(f"   Shape final após engenharia: {df.shape}")


Criando novas features...
   Shape final após engenharia: (88889, 31)


## 7. Popular Banco de Dados (Silver Layer)
Utilizamos `psycopg2.extras.execute_batch` para inserção em lote, garantindo alta performance mesmo com milhares de linhas.

In [199]:
import os
import psycopg2

print("Preparando o Banco de Dados...")

# CAMINHO CORRIGIDO: Sobe um nível (..), entra em Data_Layer, depois silver
caminho_ddl = '../Data_Layer/silver/ddl.sql'

print(f"Procurando arquivo em: {os.path.abspath(caminho_ddl)}")

if os.path.exists(caminho_ddl):
    print(f"Arquivo ENCONTRADO!")
    
    # Ler o conteúdo do arquivo SQL
    with open(caminho_ddl, 'r', encoding='utf-8') as f:
        sql_oficial = f.read()

    # Executar no Banco
    try:
        # Configuração do banco (garantindo que usa db_aviao)
        DB_CONFIG = {
            'host': 'localhost',
            'port': 5432,
            'database': 'db_aviao',
            'user': 'admin',
            'password': 'admin'
        }
        
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        
        cur.execute(sql_oficial)
        conn.commit()
        print("SUCESSO! Tabela 'aviao_silver' recriada usando o arquivo ddl.sql.")

    except Exception as e:
        print(f"Erro ao executar o SQL: {e}")
    finally:
        if 'conn' in locals(): conn.close()
        
else:
    print(f"ERRO CRÍTICO: O arquivo não está lá!")
    print("Confira se você salvou o 'ddl.sql' dentro de 'Acidentes_aviao/Data_Layer/silver/'")

Preparando o Banco de Dados...
Procurando arquivo em: c:\Users\ATA\OneDrive\Documentos\Acidentes_aviao\Data_Layer\silver\ddl.sql
Arquivo ENCONTRADO!
SUCESSO! Tabela 'aviao_silver' recriada usando o arquivo ddl.sql.


In [200]:
# --- CÉLULA DE CORREÇÃO ---
# Garante que todas as colunas necessárias existam antes de salvar

# Lista exata de colunas que o banco de dados espera
colunas_esperadas = [
    'event_id', 'investigation_type', 'accident_number', 'event_date', 
    'location', 'country', 'latitude', 'longitude', 'airport_code', 'airport_name', 
    'injury_severity', 'aircraft_damage', 'aircraft_category', 'registration_number', 
    'make', 'model', 'amateur_built', 'number_of_engines', 'engine_type', 
    'report_status', 'schedule', 'total_fatal_injuries', 'total_serious_injuries', 
    'total_minor_injuries', 'total_uninjured', 'weather_condition', 
    'broad_phase_of_flight', 'publication_date'
]

print("Verificando colunas...")
for col in colunas_esperadas:
    if col not in df.columns:
        print(f"Coluna '{col}' não encontrada. Criando vazia para evitar erros.")
        df[col] = None  # Cria a coluna preenchida com vazio
    else:
        # Garante que valores nulos (NaN) sejam convertidos para None (para o SQL aceitar)
        df[col] = df[col].replace({np.nan: None})

print("Todas as colunas estão prontas. Pode rodar a próxima célula!")

Verificando colunas...
Coluna 'schedule' não encontrada. Criando vazia para evitar erros.
Todas as colunas estão prontas. Pode rodar a próxima célula!


In [201]:
# --- CÉLULA DE LIMPEZA FINAL ---
# Remove colunas extras (como a _raw) que o banco não aceita

print(f"🧹 Removendo colunas extras...")
print(f"   Colunas antes: {len(df.columns)}")

# Mantém no DataFrame APENAS as colunas que estão na lista 'colunas_esperadas'
# (Se a lista colunas_esperadas não estiver definida, defina ela igual ao passo anterior)
colunas_finais = [c for c in colunas_esperadas if c in df.columns]
df = df[colunas_finais]

print(f"   Colunas depois: {len(df.columns)}")
print("DataFrame limpo! Agora contém apenas as colunas que o banco aceita.")

🧹 Removendo colunas extras...
   Colunas antes: 32
   Colunas depois: 28
DataFrame limpo! Agora contém apenas as colunas que o banco aceita.


In [202]:
# --- CÉLULA DE REMOÇÃO DE DUPLICATAS ---
# O banco exige IDs únicos. Vamos limpar duplicatas no CSV antes de enviar.

print(f"Verificando IDs duplicados...")
qtd_antes = len(df)

# Remove linhas onde o 'event_id' é igual, mantendo apenas a primeira aparição
df = df.drop_duplicates(subset=['event_id'], keep='first')

qtd_depois = len(df)
removidos = qtd_antes - qtd_depois

if removidos > 0:
    print(f"AVISO: Foram removidas {removidos} linhas duplicadas!")
    print(f"   (Isso resolve o erro de 'duplicate key value')")
else:
    print(" Nenhuma duplicata encontrada nos IDs.")

print(f" Total de linhas prontas para carga: {qtd_depois}")

Verificando IDs duplicados...
AVISO: Foram removidas 938 linhas duplicadas!
   (Isso resolve o erro de 'duplicate key value')
 Total de linhas prontas para carga: 87951


In [203]:
# --- CÉLULA DE AJUSTE DE TAMANHO ---
# O banco reclama se o texto for maior que o limite (50 caracteres).
# Vamos cortar o excesso por segurança.

# Lista de colunas definidas como VARCHAR(50) no  DDL
cols_limite_50 = [
    'event_id', 'investigation_type', 'accident_number', 
    'injury_severity', 'aircraft_damage', 'aircraft_category', 
    'registration_number', 'engine_type', 'schedule', 
    'weather_condition', 'report_status'
]

print(" Ajustando tamanho das strings (max 50 caracteres)...")

for col in cols_limite_50:
    if col in df.columns:
        # Converte para string e corta nos primeiros 50 caracteres
        # O 'str[:50]' pega do início até o caractere 50
        df[col] = df[col].astype(str).apply(lambda x: x[:50] if x and x != 'None' else None)

print("Textos longos foram truncados. Pronto para salvar!")

 Ajustando tamanho das strings (max 50 caracteres)...
Textos longos foram truncados. Pronto para salvar!


In [204]:
# --- CÉLULA DE CORREÇÃO DE COORDENADAS ---
# O banco aceita no máximo 4 dígitos antes da vírgula.
# Vamos anular coordenadas impossíveis (fora de -180 a 180).

print("Validando coordenadas geográficas...")

def limpar_coordenada(valor, limite):
    try:
        if valor is None:
            return None
        num = float(valor)
        # Se o número for maior que o limite (ex: latitude > 90), vira None
        if abs(num) > limite:
            return None
        return num
    except:
        return None

if 'latitude' in df.columns:
    df['latitude'] = df['latitude'].apply(lambda x: limpar_coordenada(x, 90)) # Lat vai de -90 a 90

if 'longitude' in df.columns:
    df['longitude'] = df['longitude'].apply(lambda x: limpar_coordenada(x, 180)) # Long vai de -180 a 180

print("Coordenadas inválidas foram removidas.")

Validando coordenadas geográficas...
Coordenadas inválidas foram removidas.


In [205]:
print("\nSalvando no PostgreSQL...")

# 1. Conectar ao Banco
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    print("   Conexão estabelecida.")
    
    # 2. Limpar tabela anterior (Full Refresh)
    cur.execute("TRUNCATE TABLE aviao_silver;")
    conn.commit()
    print("   Tabela truncada (limpa).")

    # 3. Preparar dados para inserção
    # Selecionamos as colunas na ordem exata da tabela do SQL
    # Garantir que a ordem aqui bata com o CREATE TABLE do ddl.sql
    
    data_values = []
    for idx, row in df.iterrows():
        data_values.append((
            str(row['event_id']),
            row['investigation_type'],
            row['accident_number'],
            row['event_date'],
            row['location'],
            row['country'],
            row['latitude'] if pd.notna(row['latitude']) else None,
            row['longitude'] if pd.notna(row['longitude']) else None,
            row['airport_code'],
            row['airport_name'],
            row['injury_severity'], # Usando a limpa
            row['aircraft_damage'],
            row['aircraft_category'],
            row['registration_number'],
            row['make'],
            row['model'],
            bool(row['amateur_built']),
            int(row['number_of_engines']),
            row['engine_type'],
            row['report_status'], 
            row['schedule'],
            # row['purpose_of_flight'],
            # row['air_carrier'],
            int(row['total_fatal_injuries']),
            int(row['total_serious_injuries']),
            int(row['total_minor_injuries']),
            int(row['total_uninjured']),
            row['weather_condition'],
            row['broad_phase_of_flight'],
            row['report_status'],
            row['publication_date']
        ))

    # ATENÇÃO: Ajuste este SQL para ter EXATAMENTE o mesmo número de %s que colunas acima
    # Vou fazer um SQL genérico seguro baseado nas colunas principais do seu DDL anterior
    insert_sql = """
        INSERT INTO aviao_silver (
            event_id, investigation_type, accident_number, event_date, location, country,
            latitude, longitude, airport_code, airport_name, injury_severity,
            aircraft_damage, aircraft_category, registration_number, make, model,
            amateur_built, number_of_engines, engine_type, far_description, schedule,
            total_fatal_injuries, total_serious_injuries, total_minor_injuries,
            total_uninjured, weather_condition, broad_phase_of_flight, report_status,
            publication_date
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
    """
    
    # OBS: O loop acima gerou 30 campos, o SQL tem 29. 
    # Precisamos garantir que bata exatamente.
    # Como o DDL pode variar, o jeito mais seguro é usar Pandas to_sql
    # Mas como o requisito é o "modelo Amazon", vamos usar o engine do SQLAlchemy que é mais robusto:
    
    engine = create_engine(DB_CONNECTION_STR)
    df.to_sql('aviao_silver', engine, if_exists='append', index=False, method='multi', chunksize=1000)
    
    print(f"   SUCESSO! {len(df)} registros inseridos na tabela aviao_silver.")

except Exception as e:
    print(f"Erro no Banco: {e}")
finally:
    if 'conn' in locals(): conn.close()


Salvando no PostgreSQL...
   Conexão estabelecida.
   Tabela truncada (limpa).
   SUCESSO! 87951 registros inseridos na tabela aviao_silver.


In [206]:
# --- CÉLULA DE VALIDAÇÃO FINAL ---
print("Espiando os dados direto do Banco de Dados...")

# Lê 5 linhas da tabela recém-criada
try:
    df_check = pd.read_sql("SELECT * FROM aviao_silver LIMIT 5", engine)
    print("Amostra dos dados na camada Silver:")
    display(df_check) # Se não funcionar display, use print(df_check)
except Exception as e:
    print(f"Erro ao ler: {e}")

Espiando os dados direto do Banco de Dados...
Amostra dos dados na camada Silver:


,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,injury_severity,aircraft_damage,aircraft_category,registration_number,make,model,amateur_built,number_of_engines,engine_type,far_description,schedule,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,None,None,Fatal,Destroyed,None,NC6404,Stinson,108-3,False,1,Reciprocating,None,None,None,None,2,0,0,0,UNK,Cruise,Probable Cause,None
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,None,None,Fatal,Destroyed,None,N5069P,Piper,PA24-180,False,1,Reciprocating,None,None,None,None,4,0,0,0,UNK,Unknown,Probable Cause,1996-09-19
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,None,None,Fatal,Destroyed,None,N5142R,Cessna,172M,False,1,Reciprocating,None,None,None,None,3,0,0,0,IMC,Cruise,Probable Cause,2007-02-26
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,None,None,Fatal,Destroyed,None,N1168J,Rockwell,112,False,1,Reciprocating,None,None,None,None,2,0,0,0,IMC,Cruise,Probable Cause,2000-09-12
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,None,None,Fatal,Destroyed,None,N15NY,Cessna,501,False,1,None,None,None,None,None,1,2,0,0,VMC,Approach,Probable Cause,1980-04-16
